In [2]:
!pip install memory_profiler

In [17]:
import numpy as np
import time
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from memory_profiler import memory_usage
import bisect

In [7]:
class PiecewiseLinearIndex:
    def __init__(self, num_segments=10):
        self.num_segments = num_segments
        self.models = []
        self.segment_boundaries = []

    def train(self, data):
        segment_size = len(data) // self.num_segments
        for i in range(self.num_segments):
            start_idx = i * segment_size
            end_idx = start_idx + segment_size if i < self.num_segments - 1 else len(data)

            segment_data = data[start_idx:end_idx]
            X = np.arange(len(segment_data)).reshape(-1, 1)
            y = segment_data

            model = LinearRegression().fit(X, y)
            self.models.append(model)
            self.segment_boundaries.append((start_idx, end_idx))

    def query(self, key):
        # Calculate segment size based on the segment boundaries
        segment_size = self.segment_boundaries[0][1] - self.segment_boundaries[0][0]

        # Determine the segment index based on the key
        segment_idx = min(key // segment_size, self.num_segments - 1)
        start_idx = self.segment_boundaries[segment_idx][0]

        # Calculate the relative position within the segment
        relative_pos = key - start_idx

        # Return the prediction for that segment using the linear model
        return self.models[segment_idx].predict([[relative_pos]])[0]

    def query_range(self, start_key, end_key):
      segment_size = self.segment_boundaries[0][1] - self.segment_boundaries[0][0]
      start_segment_idx = min(start_key // segment_size, self.num_segments - 1)
      end_segment_idx = min(end_key // segment_size, self.num_segments - 1)

      results = []
      for segment_idx in range(start_segment_idx, end_segment_idx + 1):
          start_idx = self.segment_boundaries[segment_idx][0]
          end_idx = self.segment_boundaries[segment_idx][1]

          for i in range(start_idx, end_idx):
              if start_key <= i <= end_key:
                  results.append(i)
      return results

    def update_index(self, new_data):
      combined_data = np.sort(np.concatenate([self.data, new_data]))
      self.train(combined_data)

In [8]:
class HybridIndex:
    def __init__(self, num_segments=10, hidden_layer_sizes=(50,), max_iter=1000, learning_rate_init=0.0005):
        self.num_segments = num_segments
        self.models = []
        self.nn_model = MLPRegressor(
            hidden_layer_sizes=hidden_layer_sizes,
            max_iter=max_iter,
            learning_rate_init=learning_rate_init,
            solver='adam',
            early_stopping=True,
            random_state=42
        )

    def train(self, data):
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data.reshape(-1, 1))

        X = np.arange(len(data)).reshape(-1, 1)
        y = data
        self.nn_model.fit(X, y)

    def query(self, key):
        return self.nn_model.predict([[key]])[0]

    def query_range(self, start_key, end_key):
        return [self.query(key) for key in range(start_key, end_key + 1)]

    def update_index(self, new_data):
        combined_data = np.sort(np.concatenate([self.data, new_data]))
        self.train(combined_data)

    def retrain_if_needed(self, data_update_count, threshold=100):
        if data_update_count >= threshold:
            self.train(self.data)
            data_update_count = 0

In [34]:
def generate_data(num_keys=1000):
    return np.sort(np.random.randint(1, 100000, size=num_keys))

# Measure Regular Query Performance
def measure_regular_query_performance(index, data, num_queries=1000):
    start_time = time.time()
    queries = [random.choice(data) for _ in range(num_queries)]
    results = [index.query(query) for query in queries]
    query_time = time.time() - start_time
    mse = mean_squared_error(queries, results)
    return query_time, mse

# Measure Range Query Performance
def measure_range_query_performance(index, data, num_queries=1000):
    start_time = time.time()
    queries = [(random.choice(data), random.choice(data)) for _ in range(num_queries)]

    sorted_data = np.sort(data)

    results = []
    for query in queries:
        start_idx = bisect.bisect_left(sorted_data, query[0])
        end_idx = bisect.bisect_right(sorted_data, query[1])
        results.append(sorted_data[start_idx:end_idx])

    query_time = time.time() - start_time

    true_values = [list(filter(lambda x: query[0] <= x <= query[1], data)) for query in queries]

    mse = mean_squared_error([len(true) for true in true_values], [len(result) for result in results])

    return query_time, mse

def compare_models(data):
    piecewise_index = PiecewiseLinearIndex(num_segments=10)
    hybrid_index = HybridIndex(num_segments=10)

    start_time = time.time()
    piecewise_index.train(data)
    piecewise_training_time = time.time() - start_time

    start_time = time.time()
    hybrid_index.train(data)
    hybrid_training_time = time.time() - start_time

    piecewise_regular_query_time, piecewise_regular_mse = measure_regular_query_performance(piecewise_index, data)

    hybrid_regular_query_time, hybrid_regular_mse = measure_regular_query_performance(hybrid_index, data)

    piecewise_range_query_time, piecewise_range_mse = measure_range_query_performance(piecewise_index, data)

    hybrid_range_query_time, hybrid_range_mse = measure_range_query_performance(hybrid_index, data)

    print(f"Piecewise Linear Index (BuzzDB-inspired):")
    print(f"  Training Time: {piecewise_training_time:.4f} seconds")
    print(f"  Regular Query Time (for {len(data)} queries): {piecewise_regular_query_time:.4f} seconds")
    print(f"  Mean Squared Error for Regular Queries: {piecewise_regular_mse:.4f}")
    print(f"  Range Query Time (for {len(data)} queries): {piecewise_range_query_time:.4f} seconds")

    print(f"\nHybrid Index (RMI + Neural Network):")
    print(f"  Training Time: {hybrid_training_time:.4f} seconds")
    print(f"  Regular Query Time (for {len(data)} queries): {hybrid_regular_query_time:.4f} seconds")
    print(f"  Mean Squared Error for Regular Queries: {hybrid_regular_mse:.4f}")
    print(f"  Range Query Time (for {len(data)} queries): {hybrid_range_query_time:.4f} seconds")

data = generate_data(num_keys=1000)

compare_models(data)


Piecewise Linear Index (BuzzDB-inspired):
  Training Time: 0.0085 seconds
  Regular Query Time (for 1000 queries): 0.0898 seconds
  Mean Squared Error for Regular Queries: 41762350513103.2500
  Range Query Time (for 1000 queries): 0.0035 seconds

Hybrid Index (RMI + Neural Network):
  Training Time: 4.0782 seconds
  Regular Query Time (for 1000 queries): 0.0876 seconds
  Mean Squared Error for Regular Queries: 25947715440792.2227
  Range Query Time (for 1000 queries): 0.0033 seconds
